<a id="0"></a> <br>
 # Table of Contents  
1. [Settings](#1)     
    1. [Generate directories](#2)
    1. [Define variables](#3)
    1. [Generate symbolic links](#4)
1. [Prepare the reference fasta](#5)
    1. [Index the reference fasta](#6)
    1. [Generate the contig.intervals file](#7)
    1. [Decompress the reference fasta for the probabilistic approach](#8)
1. [Pre-processing](#9)
    1. [Trimming](#10)
        1. [Trim adapters](#11)
    1. [Mapping](#12)
        1. [Map adapter-trimmed reads against the reference assembly](#13)
        1. [Sort the sam files](#14)
        1. [View the bam files](#15)
        1. [Mark duplicated reads in the bam files](#16)
        1. [Add read groups to the bam files](#17)
        1. [Index the bam files](#18)
1. [Variant calling by the consensus approach](#19)
    1. [Variant calling](#20)
        1. [Call variants](#21)
        1. [Import variants to a GenomicsDB](#22)
        1. [Joint genotyping](#23)
    1. [Derive the coverage limits for each sample](#24)
        1. [Derive coverage for each site for each sample](#25)
        1. [Calculate mean coverage across sites for each sample](#26)
        1. [Calculate the upper and lower coverage limits for each sample](#27)
    1. [Variant filtering](#28)
        1. [Derive SNP sites](#29)
        1. [Coverage filtering](#30)
            1. [Split the multi-sample vcf to single-sample vcf](#31)
            1. [Coverage filtering for each sample](#32)
            1. [Intersect to derive a multi-sample vcf for each regime](#33)
        1. [Further filtering and hard-filtering](#34)
        1. [Derive heterozygous singletons](#35)
            1. [Split GT and AD columns](#36)
            1. [Derive singletons](#37)
            1. [Filter on the allelic ratio of the mutant allele](#38)
        1. [Derive the number of candidate germline mutations for each sample](#39)
        1. [Derive bed files for manual inspection](#40)
    1. [Callability](#41)
        1. [Derive genotyped sites for each regime](#42)
        1. [Derive the number of callable sites for each regime](#43)
1. [Variant calling by the probabilistic approach](#44)
    1. [Merge bam files for each regime](#45)
    1. [Add a readgroup for the ancestor to the header of the merged bam](#46)
    1. [Call variants](#47)
    1. [Variant filtering](#48)
    1. [Derive the number of candidate germline mutations for each sample](#49)
    1. [Derive denominator](#50)
    1. [Derive bed files for manual inspection](#51)
1. [Manual inspection](#52)
    1. [Merge and annotate the bed files from both approaches for each sample](#53)
    1. [Derive single-sample vcf](#54)
    1. [Index single-sample vcf](#55)
    1. [Files needed for manual inspection](#56)
    1. [Find overlapping variants](#57)
1. [Aggregate the results](#58)

<a id="1"></a> 
# Settings

<a id="2"></a> 
## Generate directories
This section generates the directories that will be used for the analysis.

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu
FILE_DIRECTORY=/gpfs/data/justin_rna/Hwei-yen

mkdir $HOME_DIRECTORY/scripts

mkdir $SCRATCH_DIRECTORY/job_output
mkdir $SCRATCH_DIRECTORY/job_output/prepare_ref/
mkdir $SCRATCH_DIRECTORY/job_output/trimming/
mkdir $SCRATCH_DIRECTORY/job_output/bwa_mem/
mkdir $SCRATCH_DIRECTORY/job_output/samtools_sort/
mkdir $SCRATCH_DIRECTORY/job_output/samtools_view/
mkdir $SCRATCH_DIRECTORY/job_output/MarkDuplicates/
mkdir $SCRATCH_DIRECTORY/job_output/AddOrReplaceReadGroups/
mkdir $SCRATCH_DIRECTORY/job_output/samtools_index/
mkdir $SCRATCH_DIRECTORY/job_output/HaplotypeCaller/
mkdir $SCRATCH_DIRECTORY/job_output/GenomicsDBImport/
mkdir $SCRATCH_DIRECTORY/job_output/GenotypeGVCFs/
mkdir $SCRATCH_DIRECTORY/job_output/bcftools_annotate/
mkdir $SCRATCH_DIRECTORY/job_output/DP_range/
mkdir $SCRATCH_DIRECTORY/job_output/SelectVariants_SNP/
mkdir $SCRATCH_DIRECTORY/job_output/bcftools_split/
mkdir $SCRATCH_DIRECTORY/job_output/bcftools_DP/
mkdir $SCRATCH_DIRECTORY/job_output/VariantFiltration/
mkdir $SCRATCH_DIRECTORY/job_output/callability
mkdir $SCRATCH_DIRECTORY/job_output/samtools_merge/
mkdir $SCRATCH_DIRECTORY/job_output/accuMUlate/
mkdir $SCRATCH_DIRECTORY/job_output/de_novo_mutation/


mkdir $SCRATCH_DIRECTORY/ref_fasta/
mkdir $SCRATCH_DIRECTORY/fastq_files/
mkdir $SCRATCH_DIRECTORY/trimmed_seq/
mkdir $SCRATCH_DIRECTORY/bam_files/
mkdir $HOME_DIRECTORY/vcf_files/
mkdir $HOME_DIRECTORY/vcf_handling/
mkdir $HOME_DIRECTORY/vcf_handling/callability/
mkdir $HOME_DIRECTORY/vcf_handling/callability/T1/
mkdir $HOME_DIRECTORY/vcf_handling/callability/T2/
mkdir $HOME_DIRECTORY/vcf_handling/callability/T5/
mkdir $HOME_DIRECTORY/vcf_handling/GenomicsDBImport_database/
mkdir $HOME_DIRECTORY/vcf_handling/GenomicsDBImport_temp/
mkdir $HOME_DIRECTORY/vcf_handling/GenomicsDBImport_temp/T1_temp
mkdir $HOME_DIRECTORY/vcf_handling/GenomicsDBImport_temp/T2_temp
mkdir $HOME_DIRECTORY/vcf_handling/GenomicsDBImport_temp/T5_temp
mkdir $HOME_DIRECTORY/vcf_handling/GenotypeGVCFs/
mkdir $HOME_DIRECTORY/vcf_handling/bcftools_annotate/
mkdir $HOME_DIRECTORY/vcf_handling/SelectVariants_SNP/
mkdir $HOME_DIRECTORY/vcf_handling/bcftools_DP/
mkdir $HOME_DIRECTORY/vcf_handling/bcftools_DP/T1/
mkdir $HOME_DIRECTORY/vcf_handling/bcftools_DP/T2/
mkdir $HOME_DIRECTORY/vcf_handling/bcftools_DP/T5/
mkdir $HOME_DIRECTORY/vcf_handling/VariantFiltration/
mkdir $HOME_DIRECTORY/de_novo_mutation/
mkdir $HOME_DIRECTORY/de_novo_mutation/consensus_approach/
mkdir $HOME_DIRECTORY/de_novo_mutation/probabilistic_approach/
mkdir $HOME_DIRECTORY/de_novo_mutation/bed/
mkdir $HOME_DIRECTORY/de_novo_mutation/bed/manual_inspection/
mkdir $HOME_DIRECTORY/accuMUlate/

<a id="3"></a> 
## Define variables 
This section defines the variables that will be used for the analysis.

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu
FILE_DIRECTORY=/gpfs/data/justin_rna/Hwei-yen

echo "QF-1499-T1-10_S14_L008_R1_001.fastq.gz QF-1499-T1-10_S14_L008_R2_001.fastq.gz T1-10
QF-1499-T1-19_S1_L001_R1_001.fastq.gz QF-1499-T1-19_S1_L001_R2_001.fastq.gz T1-19
QF-1499-T1-37_S2_L001_R1_001.fastq.gz QF-1499-T1-37_S2_L001_R2_001.fastq.gz T1-37
QF-1499-T1-46_S15_L008_R1_001.fastq.gz QF-1499-T1-46_S15_L008_R2_001.fastq.gz T1-46
QF-1499-T1-57_S3_L001_R1_001.fastq.gz QF-1499-T1-57_S3_L001_R2_001.fastq.gz T1-57
QF-1499-T1-6_S17_L008_R1_001.fastq.gz QF-1499-T1-6_S17_L008_R2_001.fastq.gz T1-6
QF-1499-T2-1_S4_L001_R1_001.fastq.gz QF-1499-T2-1_S4_L001_R2_001.fastq.gz T2-1
QF-1499-T2-22_S6_L001_R1_001.fastq.gz QF-1499-T2-22_S6_L001_R2_001.fastq.gz T2-22
QF-1499-T2-34_S12_L008_R1_001.fastq.gz QF-1499-T2-34_S12_L008_R2_001.fastq.gz T2-34
QF-1499-T2-4_S5_L001_R1_001.fastq.gz QF-1499-T2-4_S5_L001_R2_001.fastq.gz T2-4
QF-1499-T2-42_S11_L008_R1_001.fastq.gz QF-1499-T2-42_S11_L008_R2_001.fastq.gz T2-42
QF-1499-T2-52_S7_L001_R1_001.fastq.gz QF-1499-T2-52_S7_L001_R2_001.fastq.gz T2-52
QF-1499-T5-45_S10_L008_R1_001.fastq.gz QF-1499-T5-45_S10_L008_R2_001.fastq.gz T5-45
QF-1499-T5-5_S16_L008_R1_001.fastq.gz QF-1499-T5-5_S16_L008_R2_001.fastq.gz T5-5
QF-1499-T5-70_S18_L008_R1_001.fastq.gz QF-1499-T5-70_S18_L008_R2_001.fastq.gz T5-70
QF-1499-T5-81_S13_L008_R1_001.fastq.gz QF-1499-T5-81_S13_L008_R2_001.fastq.gz T5-81
QF-1499-T5-86_S8_L001_R1_001.fastq.gz QF-1499-T5-86_S8_L001_R2_001.fastq.gz T5-86
QF-1499-T5-87_S9_L001_R1_001.fastq.gz QF-1499-T5-87_S9_L001_R2_001.fastq.gz T5-87"> $HOME_DIRECTORY/fastq_list


echo "T1
T2
T5" > $HOME_DIRECTORY/regime_list


echo "T1-10
T1-19
T1-37
T1-46
T1-57
T1-6" > $HOME_DIRECTORY/T1_sample_list

echo "$HOME_DIRECTORY/vcf_files/T1-10.g.vcf.gz
$HOME_DIRECTORY/vcf_files/T1-19.g.vcf.gz
$HOME_DIRECTORY/vcf_files/T1-37.g.vcf.gz
$HOME_DIRECTORY/vcf_files/T1-46.g.vcf.gz
$HOME_DIRECTORY/vcf_files/T1-57.g.vcf.gz
$HOME_DIRECTORY/vcf_files/T1-6.g.vcf.gz" > $HOME_DIRECTORY/T1_vcf_list

paste -d"\t" $HOME_DIRECTORY/T1_sample_list $HOME_DIRECTORY/T1_vcf_list > $HOME_DIRECTORY/T1_sample_map

echo "T2-1
T2-22
T2-34
T2-4
T2-42
T2-52" > $HOME_DIRECTORY/T2_sample_list

echo "$HOME_DIRECTORY/vcf_files/T2-1.g.vcf.gz
$HOME_DIRECTORY/vcf_files/T2-22.g.vcf.gz
$HOME_DIRECTORY/vcf_files/T2-34.g.vcf.gz
$HOME_DIRECTORY/vcf_files/T2-4.g.vcf.gz
$HOME_DIRECTORY/vcf_files/T2-42.g.vcf.gz
$HOME_DIRECTORY/vcf_files/T2-52.g.vcf.gz" > $HOME_DIRECTORY/T2_vcf_list

paste -d"\t" $HOME_DIRECTORY/T2_sample_list $HOME_DIRECTORY/T2_vcf_list > $HOME_DIRECTORY/T2_sample_map

echo "T5-45
T5-5
T5-70
T5-81
T5-86
T5-87" > $HOME_DIRECTORY/T5_sample_list

echo "$HOME_DIRECTORY/vcf_files/T5-45.g.vcf.gz
$HOME_DIRECTORY/vcf_files/T5-5.g.vcf.gz
$HOME_DIRECTORY/vcf_files/T5-70.g.vcf.gz
$HOME_DIRECTORY/vcf_files/T5-81.g.vcf.gz
$HOME_DIRECTORY/vcf_files/T5-86.g.vcf.gz
$HOME_DIRECTORY/vcf_files/T5-87.g.vcf.gz" > $HOME_DIRECTORY/T5_vcf_list

paste -d"\t" $HOME_DIRECTORY/T5_sample_list $HOME_DIRECTORY/T5_vcf_list > $HOME_DIRECTORY/T5_sample_map


printf "T1 T1-10
T1 T1-19
T1 T1-37
T1 T1-46
T1 T1-57
T1 T1-6
T2 T2-1
T2 T2-22
T2 T2-34
T2 T2-4
T2 T2-42
T2 T2-52
T5 T5-45
T5 T5-5
T5 T5-70
T5 T5-81
T5 T5-86
T5 T5-87" > $HOME_DIRECTORY/regime_sample

printf "T1 T1-10 10
T1 T1-19 11
T1 T1-37 12
T1 T1-46 13
T1 T1-57 14
T1 T1-6 15
T2 T2-1 10
T2 T2-22 11
T2 T2-34 12
T2 T2-4 13
T2 T2-42 14
T2 T2-52 15
T5 T5-45 10
T5 T5-5 11
T5 T5-70 12
T5 T5-81 13
T5 T5-86 14
T5 T5-87 15" > $HOME_DIRECTORY/regime_sample_field_for_DP

printf "T1 T1-10 0
T1 T1-19 1
T1 T1-37 2
T1 T1-46 3
T1 T1-57 4
T1 T1-6 5
T2 T2-1 0
T2 T2-22 1
T2 T2-34 2
T2 T2-4 3
T2 T2-42 4
T2 T2-52 5
T5 T5-45 0
T5 T5-5 1
T5 T5-70 2
T5 T5-81 3
T5 T5-86 4
T5 T5-87 5" > $HOME_DIRECTORY/regime_sample_field_for_callability


printf "T1 T1-10 T1-19 T1-37 T1-46 T1-57 T1-6
T2 T2-1 T2-22 T2-34 T2-4 T2-42 T2-52
T5 T5-45 T5-5 T5-70 T5-81 T5-86 T5-87" > $HOME_DIRECTORY/regime_sample_wide

echo "T1 1 2.00 60.00 3.00 40.00 -12.50 -8.00
T1 2 2.00 20.00 3.00 40.00 -12.50 -8.00
T1 3 2.00 60.00 2.00 40.00 -12.50 -8.00
T1 4 2.00 60.00 2.00 50.00 -12.50 -8.00
T1 5 2.00 60.00 3.00 40.00 -6.00 -8.00
T1 6 2.00 60.00 3.00 40.00 -12.50 -4.00
T2 1 2.00 60.00 3.00 40.00 -12.50 -8.00
T2 2 2.00 20.00 3.00 40.00 -12.50 -8.00
T2 3 2.00 60.00 2.00 40.00 -12.50 -8.00
T2 4 2.00 60.00 2.00 50.00 -12.50 -8.00
T2 5 2.00 60.00 3.00 40.00 -6.00 -8.00
T2 6 2.00 60.00 3.00 40.00 -12.50 -4.00
T5 1 2.00 60.00 3.00 40.00 -12.50 -8.00
T5 2 2.00 20.00 3.00 40.00 -12.50 -8.00
T5 3 2.00 60.00 2.00 40.00 -12.50 -8.00
T5 4 2.00 60.00 2.00 50.00 -12.50 -8.00
T5 5 2.00 60.00 3.00 40.00 -6.00 -8.00
T5 6 2.00 60.00 3.00 40.00 -12.50 -4.00" > $HOME_DIRECTORY/VariantFiltration_parameter_combinations


echo "T1 1 HOMREF_HET T1-10 T1-19 T1-37 T1-46 T1-57 T1-6
T1 1 HOMVAR_HET T1-10 T1-19 T1-37 T1-46 T1-57 T1-6
T1 2 HOMREF_HET T1-10 T1-19 T1-37 T1-46 T1-57 T1-6
T1 2 HOMVAR_HET T1-10 T1-19 T1-37 T1-46 T1-57 T1-6
T1 3 HOMREF_HET T1-10 T1-19 T1-37 T1-46 T1-57 T1-6
T1 3 HOMVAR_HET T1-10 T1-19 T1-37 T1-46 T1-57 T1-6
T1 4 HOMREF_HET T1-10 T1-19 T1-37 T1-46 T1-57 T1-6
T1 4 HOMVAR_HET T1-10 T1-19 T1-37 T1-46 T1-57 T1-6
T1 5 HOMREF_HET T1-10 T1-19 T1-37 T1-46 T1-57 T1-6
T1 5 HOMVAR_HET T1-10 T1-19 T1-37 T1-46 T1-57 T1-6
T1 6 HOMREF_HET T1-10 T1-19 T1-37 T1-46 T1-57 T1-6
T1 6 HOMVAR_HET T1-10 T1-19 T1-37 T1-46 T1-57 T1-6
T2 1 HOMREF_HET T2-1 T2-22 T2-34 T2-4 T2-42 T2-52
T2 1 HOMVAR_HET T2-1 T2-22 T2-34 T2-4 T2-42 T2-52
T2 2 HOMREF_HET T2-1 T2-22 T2-34 T2-4 T2-42 T2-52
T2 2 HOMVAR_HET T2-1 T2-22 T2-34 T2-4 T2-42 T2-52
T2 3 HOMREF_HET T2-1 T2-22 T2-34 T2-4 T2-42 T2-52
T2 3 HOMVAR_HET T2-1 T2-22 T2-34 T2-4 T2-42 T2-52
T2 4 HOMREF_HET T2-1 T2-22 T2-34 T2-4 T2-42 T2-52
T2 4 HOMVAR_HET T2-1 T2-22 T2-34 T2-4 T2-42 T2-52
T2 5 HOMREF_HET T2-1 T2-22 T2-34 T2-4 T2-42 T2-52
T2 5 HOMVAR_HET T2-1 T2-22 T2-34 T2-4 T2-42 T2-52
T2 6 HOMREF_HET T2-1 T2-22 T2-34 T2-4 T2-42 T2-52
T2 6 HOMVAR_HET T2-1 T2-22 T2-34 T2-4 T2-42 T2-52
T5 1 HOMREF_HET T5-45 T5-5 T5-70 T5-81 T5-86 T5-87
T5 1 HOMVAR_HET T5-45 T5-5 T5-70 T5-81 T5-86 T5-87
T5 2 HOMREF_HET T5-45 T5-5 T5-70 T5-81 T5-86 T5-87
T5 2 HOMVAR_HET T5-45 T5-5 T5-70 T5-81 T5-86 T5-87
T5 3 HOMREF_HET T5-45 T5-5 T5-70 T5-81 T5-86 T5-87
T5 3 HOMVAR_HET T5-45 T5-5 T5-70 T5-81 T5-86 T5-87
T5 4 HOMREF_HET T5-45 T5-5 T5-70 T5-81 T5-86 T5-87
T5 4 HOMVAR_HET T5-45 T5-5 T5-70 T5-81 T5-86 T5-87
T5 5 HOMREF_HET T5-45 T5-5 T5-70 T5-81 T5-86 T5-87
T5 5 HOMVAR_HET T5-45 T5-5 T5-70 T5-81 T5-86 T5-87
T5 6 HOMREF_HET T5-45 T5-5 T5-70 T5-81 T5-86 T5-87
T5 6 HOMVAR_HET T5-45 T5-5 T5-70 T5-81 T5-86 T5-87" > $HOME_DIRECTORY/regime_parameter_type_sample_wide



echo "T1	1	T1-10
T1	2	T1-10
T1	3	T1-10
T1	4	T1-10
T1	5	T1-10
T1	6	T1-10
T2	1	T2-1
T2	2	T2-1
T2	3	T2-1
T2	4	T2-1
T2	5	T2-1
T2	6	T2-1
T5	1	T5-45
T5	2	T5-45
T5	3	T5-45
T5	4	T5-45
T5	5	T5-45
T5	6	T5-45" > $HOME_DIRECTORY/regime_parameter_type_sample1

echo "T1	1	T1-19
T1	2	T1-19
T1	3	T1-19
T1	4	T1-19
T1	5	T1-19
T1	6	T1-19
T2	1	T2-22
T2	2	T2-22
T2	3	T2-22
T2	4	T2-22
T2	5	T2-22
T2	6	T2-22
T5	1	T5-5
T5	2	T5-5
T5	3	T5-5
T5	4	T5-5
T5	5	T5-5
T5	6	T5-5" > $HOME_DIRECTORY/regime_parameter_type_sample2

echo "T1	1	T1-37
T1	2	T1-37
T1	3	T1-37
T1	4	T1-37
T1	5	T1-37
T1	6	T1-37
T2	1	T2-34
T2	2	T2-34
T2	3	T2-34
T2	4	T2-34
T2	5	T2-34
T2	6	T2-34
T5	1	T5-70
T5	2	T5-70
T5	3	T5-70
T5	4	T5-70
T5	5	T5-70
T5	6	T5-70" > $HOME_DIRECTORY/regime_parameter_type_sample3

echo "T1	1	T1-46
T1	2	T1-46
T1	3	T1-46
T1	4	T1-46
T1	5	T1-46
T1	6	T1-46
T2	1	T2-4
T2	2	T2-4
T2	3	T2-4
T2	4	T2-4
T2	5	T2-4
T2	6	T2-4
T5	1	T5-81
T5	2	T5-81
T5	3	T5-81
T5	4	T5-81
T5	5	T5-81
T5	6	T5-81" > $HOME_DIRECTORY/regime_parameter_type_sample4

echo "T1	1	T1-57
T1	2	T1-57
T1	3	T1-57
T1	4	T1-57
T1	5	T1-57
T1	6	T1-57
T2	1	T2-42
T2	2	T2-42
T2	3	T2-42
T2	4	T2-42
T2	5	T2-42
T2	6	T2-42
T5	1	T5-86
T5	2	T5-86
T5	3	T5-86
T5	4	T5-86
T5	5	T5-86
T5	6	T5-86" > $HOME_DIRECTORY/regime_parameter_type_sample5

echo "T1	1	T1-6
T1	2	T1-6
T1	3	T1-6
T1	4	T1-6
T1	5	T1-6
T1	6	T1-6
T2	1	T2-52
T2	2	T2-52
T2	3	T2-52
T2	4	T2-52
T2	5	T2-52
T2	6	T2-52
T5	1	T5-87
T5	2	T5-87
T5	3	T5-87
T5	4	T5-87
T5	5	T5-87
T5	6	T5-87" > $HOME_DIRECTORY/regime_parameter_type_sample6


echo "theta=0.0001
mu=0.00000001
nfreqs=0.25     0.25    0.25    0.25
seq-error=0.01
ploidy-ancestor=2
ploidy-descendant=2
phi-diploid=0.001
phi-haploid=0.001
" > $HOME_DIRECTORY/accuMUlate_parameters


<a id="4"></a> 
## Generate symbolic links 
This section generates symbolic links for each sample.

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu
FILE_DIRECTORY=/gpfs/data/justin_rna/Hwei-yen

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T1-19/QF-1499-T1-19_S1_L001_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T1-19_S1_L001_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T1-19/QF-1499-T1-19_S1_L001_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T1-19_S1_L001_R2_001.fastq.gz

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T1-37/QF-1499-T1-37_S2_L001_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T1-37_S2_L001_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T1-37/QF-1499-T1-37_S2_L001_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T1-37_S2_L001_R2_001.fastq.gz

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T1-57/QF-1499-T1-57_S3_L001_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T1-57_S3_L001_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T1-57/QF-1499-T1-57_S3_L001_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T1-57_S3_L001_R2_001.fastq.gz

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T2-1/QF-1499-T2-1_S4_L001_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T2-1_S4_L001_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T2-1/QF-1499-T2-1_S4_L001_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T2-1_S4_L001_R2_001.fastq.gz

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T2-22/QF-1499-T2-22_S6_L001_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T2-22_S6_L001_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T2-22/QF-1499-T2-22_S6_L001_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T2-22_S6_L001_R2_001.fastq.gz

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T2-4/QF-1499-T2-4_S5_L001_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T2-4_S5_L001_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T2-4/QF-1499-T2-4_S5_L001_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T2-4_S5_L001_R2_001.fastq.gz

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T2-52/QF-1499-T2-52_S7_L001_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T2-52_S7_L001_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T2-52/QF-1499-T2-52_S7_L001_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T2-52_S7_L001_R2_001.fastq.gz

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T5-86/QF-1499-T5-86_S8_L001_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T5-86_S8_L001_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T5-86/QF-1499-T5-86_S8_L001_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T5-86_S8_L001_R2_001.fastq.gz

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T5-87/QF-1499-T5-87_S9_L001_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T5-87_S9_L001_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00346/Sample_QF-1499-T5-87/QF-1499-T5-87_S9_L001_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T5-87_S9_L001_R2_001.fastq.gz

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T1-10/QF-1499-T1-10_S14_L008_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T1-10_S14_L008_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T1-10/QF-1499-T1-10_S14_L008_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T1-10_S14_L008_R2_001.fastq.gz

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T1-46/QF-1499-T1-46_S15_L008_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T1-46_S15_L008_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T1-46/QF-1499-T1-46_S15_L008_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T1-46_S15_L008_R2_001.fastq.gz

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T1-6/QF-1499-T1-6_S17_L008_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T1-6_S17_L008_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T1-6/QF-1499-T1-6_S17_L008_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T1-6_S17_L008_R2_001.fastq.gz

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T2-34/QF-1499-T2-34_S12_L008_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T2-34_S12_L008_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T2-34/QF-1499-T2-34_S12_L008_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T2-34_S12_L008_R2_001.fastq.gz

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T2-42/QF-1499-T2-42_S11_L008_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T2-42_S11_L008_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T2-42/QF-1499-T2-42_S11_L008_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T2-42_S11_L008_R2_001.fastq.gz

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T5-45/QF-1499-T5-45_S10_L008_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T5-45_S10_L008_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T5-45/QF-1499-T5-45_S10_L008_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T5-45_S10_L008_R2_001.fastq.gz

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T5-5/QF-1499-T5-5_S16_L008_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T5-5_S16_L008_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T5-5/QF-1499-T5-5_S16_L008_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T5-5_S16_L008_R2_001.fastq.gz

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T5-70/QF-1499-T5-70_S18_L008_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T5-70_S18_L008_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T5-70/QF-1499-T5-70_S18_L008_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T5-70_S18_L008_R2_001.fastq.gz

ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T5-81/QF-1499-T5-81_S13_L008_R1_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T5-81_S13_L008_R1_001.fastq.gz
ln -s $FILE_DIRECTORY/nematode_data_QF1499_delivery00393/Sample_QF-1499-T5-81/QF-1499-T5-81_S13_L008_R2_001.fastq.gz $SCRATCH_DIRECTORY/fastq_files/QF-1499-T5-81_S13_L008_R2_001.fastq.gz


<a id="5"></a> 
# Prepare the reference fasta 

<a id="6"></a> 
## Index the reference fasta 

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu
FILE_DIRECTORY=/gpfs/data/justin_rna/Hwei-yen

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/prepare_ref
JOB_NAME=prepare_reference_fasta


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=3G
#SBATCH --time=0-02:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

module load bwa/0.7.17
module load java/jdk1.8.0_231
module load picard/2.24.1
module load htslib/1.15.1
module load samtools/1.16.1


bwa index -p '$SCRATCH_DIRECTORY'/ref_fasta/$(basename $REF_FASTA) '$SCRATCH_DIRECTORY'/ref_fasta/$(basename $REF_FASTA) -a bwtsw '$SCRATCH_DIRECTORY'/ref_fasta/$(basename $REF_FASTA)

java -jar /gpfs/software/ada/picard/2.24.1/picard.jar CreateSequenceDictionary -R '$SCRATCH_DIRECTORY'/ref_fasta/$(basename $REF_FASTA) -O '$SCRATCH_DIRECTORY'/ref_fasta/$(basename $REF_FASTA '.fna.gz').dict

zcat '$SCRATCH_DIRECTORY'/ref_fasta/$(basename $REF_FASTA) | bgzip -c > '$SCRATCH_DIRECTORY'/ref_fasta/$(basename $REF_FASTA)

samtools faidx '$SCRATCH_DIRECTORY'/ref_fasta/$(basename $REF_FASTA)

' > $HOME_DIRECTORY/scripts/prepare_ref.sh


# sbatch $HOME_DIRECTORY/scripts/prepare_ref.sh



<a id="7"></a> 
## Generate the contig.intervals file
This script generates the contig.intervals file that will be used in the [GenomicsDBImport](#22) step.

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu
FILE_DIRECTORY=/gpfs/data/justin_rna/Hwei-yen

REF_FASTA=GCA_001643735.4_ASM164373v4_genomic.fna.gz


cut -f1,2 $SCRATCH_DIRECTORY/ref_fasta/$(basename $REF_FASTA).fai > $SCRATCH_DIRECTORY/ref_fasta/$(basename $REF_FASTA)_contigs.temp

less $SCRATCH_DIRECTORY/ref_fasta/$(basename $REF_FASTA)_contigs.temp | while read p; do echo "1"; done > $SCRATCH_DIRECTORY/ref_fasta/$(basename $REF_FASTA)_contigs.start

paste $SCRATCH_DIRECTORY/ref_fasta/$(basename $REF_FASTA)_contigs.temp $SCRATCH_DIRECTORY/ref_fasta/$(basename $REF_FASTA)_contigs.start | awk '{ print $1,$3,$2 }' > $SCRATCH_DIRECTORY/ref_fasta/$(basename $REF_FASTA)_contigs.interval_list_1

sed -r 's/\s+/:/' $SCRATCH_DIRECTORY/ref_fasta/$(basename $REF_FASTA)_contigs.interval_list_1 > $SCRATCH_DIRECTORY/ref_fasta/$(basename $REF_FASTA)_contigs.interval_list_2

sed -r 's/\s+/-/' $SCRATCH_DIRECTORY/ref_fasta/$(basename $REF_FASTA)_contigs.interval_list_2 > $SCRATCH_DIRECTORY/ref_fasta/$(basename $REF_FASTA)_contigs.intervals


rm $SCRATCH_DIRECTORY/ref_fasta/$(basename $REF_FASTA)_contigs.temp
rm $SCRATCH_DIRECTORY/ref_fasta/$(basename $REF_FASTA)_contigs.start
rm $SCRATCH_DIRECTORY/ref_fasta/$(basename $REF_FASTA)_contigs.interval_list_1
rm $SCRATCH_DIRECTORY/ref_fasta/$(basename $REF_FASTA)_contigs.interval_list_2





<a id="8"></a> 
## Decompress the reference fasta for the probabilistic approach
This script produces a decompressed fastq for the reference assembly which will be used for the [probabilistic approach](#XXX)

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu
FILE_DIRECTORY=/gpfs/data/justin_rna/Hwei-yen

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz

zcat $SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz > $SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna

<a id="9"></a> 
# Pre-processing

<a id="10"></a> 
## Trimming

<a id="11"></a> 
### Trim adapters

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/trimming
JOB_NAME=trimming

echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=6G
#SBATCH --time=0-02:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

input_file_1=$1
input_file_2=$2

module load fastp/0.23.2

fastp -i '$SCRATCH_DIRECTORY'/fastq_files/$input_file_1 -I '$SCRATCH_DIRECTORY'/fastq_files/$input_file_2 -o '$SCRATCH_DIRECTORY'/trimmed_seq/$(basename $input_file_1 '.fastq.gz')_fastp.fastq.gz -O '$SCRATCH_DIRECTORY'/trimmed_seq/$(basename $input_file_2 '.fastq.gz')_fastp.fastq.gz --detect_adapter_for_pe -q 15 -l 15

' > $HOME_DIRECTORY/scripts/trimming.sh



less $HOME_DIRECTORY/fastq_list | while read a b c; do printf "
sbatch $HOME_DIRECTORY/scripts/trimming.sh "$a" "$b"
"; done > $HOME_DIRECTORY/scripts/trimming_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/trimming_submit_jobs


<a id="12"></a> 
## Mapping

<a id="13"></a> 
### Map adapter-trimmed reads against the reference assembly

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/bwa_mem
JOB_NAME=bwa_mem


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=18G
#SBATCH --time=0-72:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

input_file_1=$1
input_file_2=$2
output_file=$3

module load bwa/0.7.17

bwa mem -t 6 '$REF_FASTA' '$SCRATCH_DIRECTORY'/trimmed_seq/$(basename $input_file_1 '.fastq.gz')_fastp.fastq.gz '$SCRATCH_DIRECTORY'/trimmed_seq/$(basename $input_file_2 '.fastq.gz')_fastp.fastq.gz > '$SCRATCH_DIRECTORY'/trimmed_seq/$(basename $output_file).sam

' > $HOME_DIRECTORY/scripts/bwa_mem.sh



less $HOME_DIRECTORY/fastq_list | while read a b c; do printf "
sbatch $HOME_DIRECTORY/scripts/bwa_mem.sh "$a" "$b" "$c"
"; done > $HOME_DIRECTORY/scripts/bwa_mem_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/bwa_mem_submit_jobs


<a id="14"></a> 
### Sort the sam files

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/samtools_sort
JOB_NAME=samtools_sort_sam_to_bam


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=18G
#SBATCH --time=0-72:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

input_file=$1

module load samtools/1.16.1

samtools sort -T '$SCRATCH_DIRECTORY'/trimmed_seq/$(basename $input_file) -o '$SCRATCH_DIRECTORY'/bam_files/$(basename $input_file).bam '$SCRATCH_DIRECTORY'/trimmed_seq/$(basename $input_file).sam 
' > $HOME_DIRECTORY/scripts/samtools_sort_sam_to_bam.sh



less $HOME_DIRECTORY/fastq_list | while read a b c; do printf "
sbatch $HOME_DIRECTORY/scripts/samtools_sort_sam_to_bam.sh "$c"
"; done > $HOME_DIRECTORY/scripts/samtools_sort_sam_to_bam_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/samtools_sort_sam_to_bam_submit_jobs



<a id="15"></a> 
### View the bam files

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/samtools_view
JOB_NAME=samtools_view_bam


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=18G
#SBATCH --time=0-48:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

input_file=$1

module load samtools/1.16.1

samtools view '$SCRATCH_DIRECTORY'/bam_files/$(basename $input_file).bam -h -b -o '$SCRATCH_DIRECTORY'/bam_files/$(basename $input_file)_viewed.bam

' > $HOME_DIRECTORY/scripts/samtools_view_bam.sh



less $HOME_DIRECTORY/fastq_list | while read a b c; do printf "
sbatch $HOME_DIRECTORY/scripts/samtools_view_bam.sh "$c"
"; done > $HOME_DIRECTORY/scripts/samtools_view_bam_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/samtools_view_bam_submit_jobs



<a id="16"></a> 
### Mark duplicated reads in the bam files

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/MarkDuplicates
JOB_NAME=MarkDuplicates


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=36G
#SBATCH --time=0-02:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

input_file=$1

module load java/jdk1.8.0_231
module load picard/2.24.1

java -jar /gpfs/software/ada/picard/2.24.1/picard.jar MarkDuplicates I='$SCRATCH_DIRECTORY'/bam_files/$(basename $input_file)_viewed.bam O='$SCRATCH_DIRECTORY'/bam_files/$(basename $input_file)_viewed_marked.bam METRICS_FILE='$SCRATCH_DIRECTORY'/bam_files/$(basename $input_file)_metrics.txt ASSUME_SORTED=true VALIDATION_STRINGENCY=SILENT

' > $HOME_DIRECTORY/scripts/MarkDuplicates.sh



less $HOME_DIRECTORY/fastq_list | while read a b c; do printf "
sbatch $HOME_DIRECTORY/scripts/MarkDuplicates.sh "$c"
"; done > $HOME_DIRECTORY/scripts/MarkDuplicates_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/MarkDuplicates_submit_jobs



<a id="17"></a> 
### Add read groups to the bam files

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/AddOrReplaceReadGroups
JOB_NAME=AddOrReplaceReadGroups


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=6G
#SBATCH --time=0-12:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

input_file=$1

module load java/jdk1.8.0_231
module load picard/2.24.1

java -jar /gpfs/software/ada/picard/2.24.1/picard.jar AddOrReplaceReadGroups I='$SCRATCH_DIRECTORY'/bam_files/$(basename $input_file)_viewed_marked.bam  O='$SCRATCH_DIRECTORY'/bam_files/$(basename $input_file)_viewed_marked_addg.bam RGID=$(basename $input_file) RGLB=lib1 RGPL=illumina_TruSeq RGSM=$(basename $input_file) RGPU=unit1

' > $HOME_DIRECTORY/scripts/AddOrReplaceReadGroups.sh



less $HOME_DIRECTORY/fastq_list | while read a b c; do printf "
sbatch $HOME_DIRECTORY/scripts/AddOrReplaceReadGroups.sh "$c"
"; done > $HOME_DIRECTORY/scripts/AddOrReplaceReadGroups_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/AddOrReplaceReadGroups_submit_jobs





<a id="18"></a> 
### Index the bam files

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/samtools_index
JOB_NAME=samtools_index_bam


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=6G
#SBATCH --time=0-01:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

input_file=$1

module load samtools/1.16.1

samtools index '$SCRATCH_DIRECTORY'/bam_files/$(basename $input_file)_viewed_marked_addg.bam 

' > $HOME_DIRECTORY/scripts/samtools_index.sh



less $HOME_DIRECTORY/fastq_list | while read a b c; do printf "
sbatch $HOME_DIRECTORY/scripts/samtools_index.sh "$c"
"; done > $HOME_DIRECTORY/scripts/samtools_index_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/samtools_index_submit_jobs



<a id="19"></a> 
# Variant calling by the consensus approach

<a id="20"></a> 
## Variant calling

<a id="21"></a> 
### Call variants
This script uses the GATK HaplotypeCaller tool to generate a single-sample vcf for each sample.

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/HaplotypeCaller
JOB_NAME=HaplotypeCaller


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=30G
#SBATCH --time=6-00:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

input_file=$1

module load java/jdk1.8.0_231
module load GATK/4.9.1

gatk HaplotypeCaller -R '$REF_FASTA' -I '$SCRATCH_DIRECTORY'/bam_files/$(basename $input_file)_viewed_marked_addg.bam -O '$HOME_DIRECTORY'/vcf_files/$(basename $input_file).g.vcf.gz -ploidy 2 -ERC BP_RESOLUTION --dont-use-soft-clipped-bases

' > $HOME_DIRECTORY/scripts/HaplotypeCaller.sh



less $HOME_DIRECTORY/fastq_list | while read a b c; do printf "
sbatch $HOME_DIRECTORY/scripts/HaplotypeCaller.sh "$c"
"; done > $HOME_DIRECTORY/scripts/HaplotypeCaller_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/HaplotypeCaller_submit_jobs


<a id="22"></a> 
### Import variants to a GenomicsDB
This script uses the GATK GenomicsDBImport tool to import single-sample vcfs into a GenomicsDB before joint genotyping. Specifically, single-sample vcfs of the same regime are imported into a GenomicsDB (six single-sample vcfs per regime); total three GenomicsDBs, corresponding to the three regimes (T1, T2, T5), are generated.

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
REF_FASTA_INTERVALS=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz_contigs.intervals
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/GenomicsDBImport
JOB_NAME=GenomicsDBImport


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=80G
#SBATCH --time=0-24:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

regime=$1

module load java/jdk1.8.0_231
module load GATK/4.9.1

gatk GenomicsDBImport --genomicsdb-workspace-path '$HOME_DIRECTORY'/vcf_handling/GenomicsDBImport_database/$(basename $regime)_database --tmp-dir '$HOME_DIRECTORY'/vcf_handling/GenomicsDBImport_temp/$(basename $regime)_temp --batch-size 10 --sample-name-map '$HOME_DIRECTORY'/$(basename $regime)_sample_map --reader-threads 10 -L '$REF_FASTA_INTERVALS' --showHidden True --merge-input-intervals True

' > $HOME_DIRECTORY/scripts/GenomicsDBImport.sh



less $HOME_DIRECTORY/regime_list | while read a; do printf "
sbatch $HOME_DIRECTORY/scripts/GenomicsDBImport.sh "$a"
"; done > $HOME_DIRECTORY/scripts/GenomicsDBImport_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/GenomicsDBImport_submit_jobs


<a id="23"></a> 
### Joint genotyping
This script uses the GATK GenotypeGVCFs tool to performe joint genotype and produces a multi-sample vcf for each GenomicsDB (i.e. for each regime).

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/GenotypeGVCFs
JOB_NAME=GenotypeGVCFs


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=30G
#SBATCH --time=0-24:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

regime=$1

module load java/jdk1.8.0_231
module load GATK/4.9.1

gatk GenotypeGVCFs -R '$REF_FASTA' -V gendb://'$HOME_DIRECTORY'/vcf_handling/GenomicsDBImport_database/$(basename $regime)_database -G StandardAnnotation -O '$HOME_DIRECTORY'/vcf_handling/GenotypeGVCFs/$(basename $regime)_GenotypeGVCFs.g.vcf.gz --include-non-variant-sites

' > $HOME_DIRECTORY/scripts/GenotypeGVCFs.sh



less $HOME_DIRECTORY/regime_list | while read a; do printf "
sbatch $HOME_DIRECTORY/scripts/GenotypeGVCFs.sh "$a"
"; done > $HOME_DIRECTORY/scripts/GenotypeGVCFs_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/GenotypeGVCFs_submit_jobs



<a id="24"></a> 
## Derive the coverage limits for each sample
This section calculates the mean coverage, as well as the uppler and lower coverage limits for each sample to facilitate [coverage filtering](#30).

<a id="25"></a> 
### Derive coverage for each site for each sample
This script uses the bcftools annotate tool to derive the coverage (FORMAT/DP field) for each site for each sample

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/bcftools_annotate
JOB_NAME=bcftools_annotate


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=6G
#SBATCH --time=0-24:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

regime=$1

module load bcftools/1.15.1

bcftools annotate -x ^FORMAT/DP '$HOME_DIRECTORY'/vcf_handling/GenotypeGVCFs/$(basename $regime)_GenotypeGVCFs.g.vcf.gz -o '$HOME_DIRECTORY'/vcf_handling/bcftools_annotate/$(basename $regime)_bcftools_annotate_DP

' > $HOME_DIRECTORY/scripts/bcftools_annotate.sh



less $HOME_DIRECTORY/regime_list | while read a; do printf "
sbatch $HOME_DIRECTORY/scripts/bcftools_annotate.sh "$a"
"; done > $HOME_DIRECTORY/scripts/bcftools_annotate_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/bcftools_annotate_submit_jobs


<a id="26"></a> 
### Calculate mean coverage across sites for each sample

In [ ]:
## Derive mean DP for each sample

HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/DP_range
JOB_NAME=DP_range

echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=3G
#SBATCH --time=0-12:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

regime=$1
sample_ID=$2
field=$3

sed '/^#/d' '$HOME_DIRECTORY'/vcf_handling/bcftools_annotate/$(basename $regime)_bcftools_annotate_DP | cut -f$field > '$HOME_DIRECTORY'/vcf_handling/bcftools_annotate/$(basename $sample_ID)_bcftools_annotate_DP 

awk '"'"'{ total += $1 } END { print total/NR }'"'"' '$HOME_DIRECTORY'/vcf_handling/bcftools_annotate/$(basename $sample_ID)_bcftools_annotate_DP  > '$HOME_DIRECTORY'/vcf_handling/bcftools_annotate/$(basename $sample_ID)_bcftools_annotate_DP_mean

' > $HOME_DIRECTORY/scripts/DP_range.sh


less $HOME_DIRECTORY/regime_sample_field_for_DP | while read a b c; do printf "
sbatch $HOME_DIRECTORY/scripts/DP_range.sh "$a" "$b" "$c"
"; done > $HOME_DIRECTORY/scripts/DP_range_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/DP_range_submit_jobs


<a id="27"></a> 
### Calculate the upper and lower coverage limits for each sample
This script calculates the upper (2 * mean coverage) and the lower (0.5 * mean coverage) limits for each sample, convert the numeric values to integer, and store the results in a table.

**Interperating the table:** 

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

cat $HOME_DIRECTORY/vcf_handling/bcftools_annotate/*mean | paste $HOME_DIRECTORY/regime_sample_field_for_DP - > $HOME_DIRECTORY/vcf_handling/bcftools_annotate/DP_range

awk '{$5=$4*0.5; $6=$4*2; print $0}' $HOME_DIRECTORY/vcf_handling/bcftools_annotate/DP_range > $HOME_DIRECTORY/vcf_handling/bcftools_annotate/DP_range_temp && mv $HOME_DIRECTORY/vcf_handling/bcftools_annotate/DP_range_temp $HOME_DIRECTORY/DP_range

awk '{$7=int($5); $8=int($6); print $0}' $HOME_DIRECTORY/DP_range > $HOME_DIRECTORY/DP_range_temp && mv $HOME_DIRECTORY/DP_range_temp $HOME_DIRECTORY/DP_range


<a id="28"></a> 
## Variant filtering

<a id="29"></a> 
### Derive SNP sites
This script uses the GATK SelectVariants tool to select sites that are SNPs.

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/SelectVariants_SNP
JOB_NAME=SelectVariants_SNP


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=12G
#SBATCH --time=00-06:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

regime=$1

module load java/jdk1.8.0_231
module load GATK/4.9.1

gatk SelectVariants -R '$REF_FASTA' -V '$HOME_DIRECTORY'/vcf_handling/GenotypeGVCFs/$(basename $regime)_GenotypeGVCFs.g.vcf.gz -O '$HOME_DIRECTORY'/vcf_handling/SelectVariants_SNP/$(basename $regime)_SelectVariants_SNP.g.vcf.gz --select-type-to-include SNP --select-type-to-include NO_VARIATION --exclude-non-variants false
' > $HOME_DIRECTORY/scripts/SelectVariants_SNP.sh



less $HOME_DIRECTORY/regime_list | while read a; do printf "
sbatch $HOME_DIRECTORY/scripts/SelectVariants_SNP.sh "$a"
"; done > $HOME_DIRECTORY/scripts/SelectVariants_SNP_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/SelectVariants_SNP_submit_jobs



<a id="30"></a> 
### Coverage filtering
This script filters to derive SNPs with coverages within the upper or lower coverage limits for each sample.

<a id="31"></a> 
#### Split the multi-sample vcf to single-sample vcf
This script uses the bcftools split tool to split the multi-sample vcf containing SNPs for each regime (i.e. the output from [SelectVariants](#29)) to six single-sample vcf; total 18 single-sample vcf files (six per regime) are generated.

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/bcftools_split
JOB_NAME=bcftools_split

echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=3G
#SBATCH --time=0-04:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

regime=$1
sample_ID=$2

module load bcftools/1.15.1
module load htslib/1.15.1

bcftools +split '$HOME_DIRECTORY'/vcf_handling/SelectVariants_SNP/$(basename $regime)_SelectVariants_SNP.g.vcf.gz -O z -o '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/

tabix -p vcf '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $sample_ID).vcf.gz

' > $HOME_DIRECTORY/scripts/bcftools_split.sh


less $HOME_DIRECTORY/regime_sample_field_for_DP | while read a b c; do printf "
sbatch $HOME_DIRECTORY/scripts/bcftools_split.sh "$a" "$b"
"; done > $HOME_DIRECTORY/scripts/bcftools_split_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/bcftools_split_submit_jobs


<a id="32"></a> 
#### Coverage filtering for each sample
This script uses the bcftools view tool to derive sites that pass the sample's coverage filters for each sample (i.e. sites with coverages within 0.5* and 2* of the mean coverage of the sample are kept; see [Calculate the upper and lower coverage limits for each sample](#27)). The output of this script is a single-sample vcf for each sample.

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/bcftools_DP
JOB_NAME=bcftools_view_DP


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=3G
#SBATCH --time=0-02:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

regime=$1
sample_ID=$2
min_DP=$3
max_DP=$4

module load bcftools/1.15.1
module load htslib/1.15.1

bcftools view -i '"'"'FMT/DP > '"'"'$min_DP'"'"'  & FMT/DP < '"'"'$max_DP'"'"' '"'"' '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $sample_ID).vcf.gz -o '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $sample_ID)_DP.vcf.gz

tabix -p vcf '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $sample_ID)_DP.vcf.gz

' > $HOME_DIRECTORY/scripts/bcftools_view_DP.sh


less $HOME_DIRECTORY/DP_range | while read a b c d e f g h; do printf "
sbatch $HOME_DIRECTORY/scripts/bcftools_view_DP.sh "$a" "$b" "$g" "$h"
"; done > $HOME_DIRECTORY/scripts/bcftools_view_DP_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/bcftools_view_DP_submit_jobs


<a id="33"></a> 
#### Intersect to derive a multi-sample vcf for each regime
This script uses the bcftools isec tool to generate a multi-sample vcf for each regime containg SNPs that pass the sample-specific coverage filters for all six samples within the regime; total three multi-sample vcf files, corresponding to the three regimes (T1, T2, T5), are generated.

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/bcftools_DP
JOB_NAME=bcftools_intersect


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=18G
#SBATCH --time=0-8:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

regime=$1
sample1=$2
sample2=$3
sample3=$4
sample4=$5
sample5=$6
sample6=$7

module load bcftools/1.15.1
module load htslib/1.15.1


# 1. generate the intersection within regime for each regime; this produces a multi-sample vcf containing sites that passed the sample-specific DP filters for all samples within a regime

bcftools isec -n=6 -w1 '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $sample1)_DP.vcf.gz '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $sample2)_DP.vcf.gz '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $sample3)_DP.vcf.gz '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $sample4)_DP.vcf.gz '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $sample5)_DP.vcf.gz '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $sample6)_DP.vcf.gz -o '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $regime)_bcftools_isec.vcf.gz -O z

tabix -p vcf '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $regime)_bcftools_isec.vcf.gz

# 2. generate the intersection between 1 and the multi-sample vcf containing SNPs generated by SelectVariants

bcftools isec -n=2 -w1 '$HOME_DIRECTORY'/vcf_handling/SelectVariants_SNP/$(basename $regime)_SelectVariants_SNP.g.vcf.gz '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $regime)_bcftools_isec.vcf.gz -o '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $regime)_SNP_DPfiltered.vcf.gz -O z

tabix -p vcf '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $regime)_SNP_DPfiltered.vcf.gz

' > $HOME_DIRECTORY/scripts/bcftools_intersect.sh


less $HOME_DIRECTORY/regime_sample_wide | while read a b c d e f g; do printf "
sbatch $HOME_DIRECTORY/scripts/bcftools_intersect.sh "$a" "$b" "$c" "$d" "$e" "$f" "$g"
"; done > $HOME_DIRECTORY/scripts/bcftools_intersect_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/bcftools_intersect_submit_jobs



<a id="34"></a> 
### Further filtering and hard-filtering
This script uses the multi-sample vcf for each regime generated from the [coverage filtering step](#33) as input and further filters to derive biallelic sites where all samples of the same regime are genotyped (Step 1; see below) and for sites that pass the five hard-filter combinations (Step 2; see below), and outputs a tab-delimited table for each hard-filter combination (Step 3). Total 15 tables, five for each regime, are generated.

Specifically, the three steps are:
**Step 1:** Use GATK SelectVariants tool to derive biallelic sites where all samples of the same regime are genotyped. This step uses the multi-sample vcf for each regime from the [coverage filtering step](#33) and outputs a multi-sample vcf for each regime for Step 2.
**Step 2:** Use GATK VariantFiltration and bcftools view tools to derive sites that pass the hard-filters. Total five hard-filter combinations are designed (defined [here](#3)). This step uses the multi-sample vcf generated in Step 1 as input and outputs five resultant multi-sample vcf files for each input vcf, with each of the resultant vcf corresponding to a hard-filter combination.
**Step 3:** Use GATK VariantsToTable tool to extract INFO and FORMAT fields from the input vcf and outputs a tab-delimited table. This step uses the multi-sample vcf generated in Step 2 as input and outputs a tab-delimited table for each input vcf. Total five tables, each corresponding to a hard-filter combination, are generated for each regime. 

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/VariantFiltration
JOB_NAME=VariantFiltration


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=12G
#SBATCH --time=0-06:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB


regime=$1
parameter_combination=$2
QD_threshold=$3
FS_threshold=$4
SOR_threshold=$5
MQ_threshold=$6
MQRankSum_threshold=$7
ReadPosRankSum_threshold=$8


module load java/jdk1.8.0_231
module load GATK/4.9.1


# 1. Filters to derive biallelic SNP sites where all samples within the regime are called

gatk SelectVariants -R '$REF_FASTA' \
-V '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $regime)_SNP_DPfiltered.vcf.gz \
--max-nocall-number 0 \
--restrict-alleles-to BIALLELIC \
-O '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC.vcf

# 2. Filters to derive sites that passed the hard-filters

gatk VariantFiltration -R '$REF_FASTA' \
-V '$HOME_DIRECTORY'/vcf_handling/bcftools_DP/$(basename $regime)/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC.vcf \
-O '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination).vcf.gz \
--filter "QD < $QD_threshold" \
--filter-name "QD_threshold" \
--filter "FS > $FS_threshold" \
--filter-name "FS_threshold" \
--filter "SOR > $SOR_threshold" \
--filter-name "SOR_threshold" \
--filter "MQ < $MQ_threshold" \
--filter-name "MQ_threshold" \
--filter "MQRankSum < $MQRankSum_threshold" \
--filter-name "MQRankSum_threshold" \
--filter "ReadPosRankSum < $ReadPosRankSum_threshold" \
--filter-name "ReadPosRankSum_threshold" 

module load bcftools/1.15.1
module load htslib/1.15.1

bcftools view -i '"'"'FILTER="PASS"'"'"' '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination).vcf.gz -o '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination)_PASS.vcf.gz

tabix -p vcf '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination)_PASS.vcf.gz


# 3. VariantsToTable

gatk VariantsToTable -R '$REF_FASTA' \
-V '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination)_PASS.vcf.gz \
-O '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination)_PASS.table \
-F CHROM -F POS -F REF -F ALT -F QD -F QUAL -F DP \
-F MQRankSum -F ReadPosRankSum -F MQ -F FS -F SOR -F AC -F AF -F AN -F HOM-REF -F HET -F HOM-VAR -F NO-CALL -F TRANSITION -F TYPE -F VAR \
-GF GT -GF AD -GF DP -GF GQ 

' > $HOME_DIRECTORY/scripts/VariantFiltration.sh



less $HOME_DIRECTORY/VariantFiltration_parameter_combinations | while read a b c d e f g h; do printf "
sbatch $HOME_DIRECTORY/scripts/VariantFiltration.sh "$a" "$b" "$c" "$d" "$e" "$f" "$g" "$h"
"; done > $HOME_DIRECTORY/scripts/VariantFiltration_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/VariantFiltration_submit_jobs



<a id="35"></a> 
### Derive heterozygous singletons
This section uses the tables generated in the [hard-filtering step](#34) as input and further filters to derive sites where the mutant allele is found in only one of all samples in each regime (i.e. singletons), sites where the mutant sample is heterozygous and the non-mutant samples are homozygous (for the same allele, either reference or alternate), and site where the alleleic ratio of the mutant allele is between 0.25 and 0.75 in the mutant sample, and 0 in the non-mutant samples of the same regime.

Specifically, this section contains three scripts:
**[Script 1:](#36)** Split the GT and AD columns to facilitate filtering.
**[Script 2:](#37)** Derive singletons. Two types of singltons are derived; the mutant sample is heterozygous and the non-mutant samples are homozygous for the reference allele, or the mutant sample is heterozygous and the non-mutant samples are homozygous for the alternate allele.
**[Script 3:](#38)** Derive sites where the allelic ratio of the mutant allele is between 0.25 and 0.75 in the mutant sample, and 0 in the non-mutant samples of the same regime.

<a id="36"></a> 
#### Split GT and AD columns

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/VariantFiltration
JOB_NAME=split_columns


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=3G
#SBATCH --time=0-01:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB


regime=$1
parameter_combination=$2


## split GT columns

sed '"'"'s/|/\t/g'"'"' '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination)_PASS.table > '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination)_PASS_temp1.table 

sed '"'"'s/\//\t/g'"'"' '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination)_PASS_temp1.table > '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination)_PASS_temp2.table 

## split AD columns

sed '"'"'s/,/\t/g'"'"' '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination)_PASS_temp2.table > '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination)_PASS_splited.table 

rm '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination)_PASS_temp1.table
rm '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination)_PASS_temp2.table

' > $HOME_DIRECTORY/scripts/split_columns.sh



less $HOME_DIRECTORY/VariantFiltration_parameter_combinations | while read a b c d e f g h; do printf "
sbatch $HOME_DIRECTORY/scripts/split_columns.sh "$a" "$b"
"; done > $HOME_DIRECTORY/scripts/split_columns_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/split_columns_submit_jobs


<a id="37"></a> 
#### Derive singletons

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/VariantFiltration
JOB_NAME=singletons


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=3G
#SBATCH --time=0-01:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB


regime=$1
parameter_combination=$2

## HOM-REF == 5, HET == 1

awk '"'"'($16 == 5) && ($17 == 1) && ($18 == 0) {print $0}'"'"' '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination)_PASS_splited.table > '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination)_PASS_splited_singletons_HOMREF_HET.table

## HOM-VAR == 5, HET == 1

awk '"'"'($16 == 0) && ($17 == 1) && ($18 == 5) {print $0}'"'"' '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination)_PASS_splited.table > '$HOME_DIRECTORY'/vcf_handling/VariantFiltration/$(basename $regime)_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_$(basename $parameter_combination)_PASS_splited_singletons_HOMVAR_HET.table

' > $HOME_DIRECTORY/scripts/singletons.sh



less $HOME_DIRECTORY/VariantFiltration_parameter_combinations | while read a b c d e f g h; do printf "
sbatch $HOME_DIRECTORY/scripts/singletons.sh "$a" "$b"
"; done > $HOME_DIRECTORY/scripts/singletons_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/singletons_submit_jobs



<a id="38"></a> 
#### Filter on the allelic ratio of the mutant allele
This script performs further filtering to derive sites where the allelic ratio of mutant allele is between 0.25 and 0.75 in the mutant sample, and no mutant allele is found in non-mutant samples of the same regime.


In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu


less $HOME_DIRECTORY/regime_parameter_type_sample1 | while read a b c; do printf "
## for sample1, HOM-REF/HET
awk ' (\$23 == \$3) && (\$24 == \$4) && (\$29 == \$3) && (\$30 == \$3) && (\$35 == \$3) && (\$36 == \$3) && (\$41 == \$3) && (\$42 == \$3) && (\$47 == \$3) && (\$48 == \$3) && (\$53 == \$3) && (\$54 == \$3) && (\$26/(\$26+\$25) >= 0.25) && (\$26/(\$26+\$25) <= 0.75) && (\$32 == 0) && (\$38 == 0) && (\$44 == 0) && (\$50 == 0) && (\$56 == 0) ' $HOME_DIRECTORY/vcf_handling/VariantFiltration/"$a"_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_"$b"_PASS_splited_singletons_HOMREF_HET.table > $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$c"_"$b"_HOMREF_HET.table
## for sample1, HOM-VAR/HET
awk ' (\$23 == \$3) && (\$24 == \$4) && (\$29 == \$4) && (\$30 == \$4) && (\$35 == \$4) && (\$36 == \$4) && (\$41 == \$4) && (\$42 == \$4) && (\$47 == \$4) && (\$48 == \$4) && (\$53 == \$4) && (\$54 == \$4) && (\$26/(\$26+\$25) >= 0.25) && (\$26/(\$26+\$25) <= 0.75) && (\$31 == 0) && (\$37 == 0) && (\$43 == 0) && (\$49 == 0) && (\$55 == 0) ' $HOME_DIRECTORY/vcf_handling/VariantFiltration/"$a"_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_"$b"_PASS_splited_singletons_HOMVAR_HET.table > $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$c"_"$b"_HOMVAR_HET.table
"; done > $HOME_DIRECTORY/scripts/singletons_filtering_sample1




less $HOME_DIRECTORY/regime_parameter_type_sample2 | while read a b c; do printf "
## for sample2, HOM-REF/HET
awk ' (\$23 == \$3) && (\$24 == \$3) && (\$29 == \$3) && (\$30 == \$4) && (\$35 == \$3) && (\$36 == \$3) && (\$41 == \$3) && (\$42 == \$3) && (\$47 == \$3) && (\$48 == \$3) && (\$53 == \$3) && (\$54 == \$3) && (\$32/(\$32+\$31) >= 0.25) && (\$32/(\$32+\$31) <= 0.75) && (\$26 == 0) && (\$38 == 0) && (\$44 == 0) && (\$50 == 0) && (\$56 == 0) ' $HOME_DIRECTORY/vcf_handling/VariantFiltration/"$a"_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_"$b"_PASS_splited_singletons_HOMREF_HET.table > $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$c"_"$b"_HOMREF_HET.table
## for sample2, HOM-VAR/HET
awk ' (\$23 == \$4) && (\$24 == \$4) && (\$29 == \$3) && (\$30 == \$4) && (\$35 == \$4) && (\$36 == \$4) && (\$41 == \$4) && (\$42 == \$4) && (\$47 == \$4) && (\$48 == \$4) && (\$53 == \$4) && (\$54 == \$4) && (\$32/(\$32+\$31) >= 0.25) && (\$32/(\$32+\$31) <= 0.75) && (\$25 == 0) && (\$37 == 0) && (\$43 == 0) && (\$49 == 0) && (\$55 == 0) ' $HOME_DIRECTORY/vcf_handling/VariantFiltration/"$a"_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_"$b"_PASS_splited_singletons_HOMVAR_HET.table > $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$c"_"$b"_HOMVAR_HET.table
"; done > $HOME_DIRECTORY/scripts/singletons_filtering_sample2



less $HOME_DIRECTORY/regime_parameter_type_sample3 | while read a b c; do printf "
## for sample3, HOM-REF/HET
awk ' (\$23 == \$3) && (\$24 == \$3) && (\$29 == \$3) && (\$30 == \$3) && (\$35 == \$3) && (\$36 == \$4) && (\$41 == \$3) && (\$42 == \$3) && (\$47 == \$3) && (\$48 == \$3) && (\$53 == \$3) && (\$54 == \$3) && (\$38/(\$38+\$37) >= 0.25) && (\$38/(\$38+\$37) <= 0.75) && (\$26 == 0) && (\$32 == 0) && (\$44 == 0) && (\$50 == 0) && (\$56 == 0) ' $HOME_DIRECTORY/vcf_handling/VariantFiltration/"$a"_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_"$b"_PASS_splited_singletons_HOMREF_HET.table > $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$c"_"$b"_HOMREF_HET.table
## for sample3, HOM-VAR/HET
awk ' (\$23 == \$4) && (\$24 == \$4) && (\$29 == \$4) && (\$30 == \$4) && (\$35 == \$3) && (\$36 == \$4) && (\$41 == \$4) && (\$42 == \$4) && (\$47 == \$4) && (\$48 == \$4) && (\$53 == \$4) && (\$54 == \$4) && (\$38/(\$38+\$37) >= 0.25) && (\$38/(\$38+\$37) <= 0.75) && (\$25 == 0) && (\$31 == 0) && (\$43 == 0) && (\$49 == 0) && (\$55 == 0) ' $HOME_DIRECTORY/vcf_handling/VariantFiltration/"$a"_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_"$b"_PASS_splited_singletons_HOMVAR_HET.table > $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$c"_"$b"_HOMVAR_HET.table
"; done > $HOME_DIRECTORY/scripts/singletons_filtering_sample3




less $HOME_DIRECTORY/regime_parameter_type_sample4 | while read a b c; do printf "
## for sample4, HOM-REF/HET
awk ' (\$23 == \$3) && (\$24 == \$3) && (\$29 == \$3) && (\$30 == \$3) && (\$35 == \$3) && (\$36 == \$3) && (\$41 == \$3) && (\$42 == \$4) && (\$47 == \$3) && (\$48 == \$3) && (\$53 == \$3) && (\$54 == \$3) && (\$44/(\$44+\$43) >= 0.25) && (\$44/(\$44+\$43) <= 0.75) && (\$26 == 0) && (\$32 == 0) && (\$38 == 0) && (\$50 == 0) && (\$56 == 0) ' $HOME_DIRECTORY/vcf_handling/VariantFiltration/"$a"_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_"$b"_PASS_splited_singletons_HOMREF_HET.table > $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$c"_"$b"_HOMREF_HET.table
## for sample4, HOM-VAR/HET
awk ' (\$23 == \$4) && (\$24 == \$4) && (\$29 == \$4) && (\$30 == \$4) && (\$35 == \$4) && (\$36 == \$4) && (\$41 == \$3) && (\$42 == \$4) && (\$47 == \$4) && (\$48 == \$4) && (\$53 == \$4) && (\$54 == \$4) && (\$44/(\$44+\$43) >= 0.25) && (\$44/(\$44+\$43) <= 0.75) && (\$25 == 0) && (\$31 == 0) && (\$37 == 0) && (\$49 == 0) && (\$55 == 0) ' $HOME_DIRECTORY/vcf_handling/VariantFiltration/"$a"_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_"$b"_PASS_splited_singletons_HOMVAR_HET.table > $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$c"_"$b"_HOMVAR_HET.table
"; done > $HOME_DIRECTORY/scripts/singletons_filtering_sample4




less $HOME_DIRECTORY/regime_parameter_type_sample5 | while read a b c; do printf "
## for sample5, HOM-REF/HET
awk ' (\$23 == \$3) && (\$24 == \$3) && (\$29 == \$3) && (\$30 == \$3) && (\$35 == \$3) && (\$36 == \$3) && (\$41 == \$3) && (\$42 == \$3) && (\$47 == \$3) && (\$48 == \$4) && (\$53 == \$3) && (\$54 == \$3) && (\$50/(\$49+\$50) >= 0.25) && (\$50/(\$49+\$50) <= 0.75) && (\$26 == 0) && (\$32 == 0) && (\$38 == 0) && (\$44 == 0) && (\$56 == 0) ' $HOME_DIRECTORY/vcf_handling/VariantFiltration/"$a"_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_"$b"_PASS_splited_singletons_HOMREF_HET.table > $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$c"_"$b"_HOMREF_HET.table
## for sample5, HOM-VAR/HET
awk ' (\$23 == \$4) && (\$24 == \$4) && (\$29 == \$4) && (\$30 == \$4) && (\$35 == \$4) && (\$36 == \$4) && (\$41 == \$4) && (\$42 == \$4) && (\$47 == \$3) && (\$48 == \$4) && (\$53 == \$4) && (\$54 == \$4) && (\$50/(\$49+\$50) >= 0.25) && (\$50/(\$49+\$50) <= 0.75) && (\$25 == 0) && (\$31 == 0) && (\$37 == 0) && (\$43 == 0) && (\$55 == 0) ' $HOME_DIRECTORY/vcf_handling/VariantFiltration/"$a"_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_"$b"_PASS_splited_singletons_HOMVAR_HET.table > $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$c"_"$b"_HOMVAR_HET.table
"; done > $HOME_DIRECTORY/scripts/singletons_filtering_sample5



less $HOME_DIRECTORY/regime_parameter_type_sample6 | while read a b c; do printf "
## for sample6, HOM-REF/HET
awk ' (\$23 == \$3) && (\$24 == \$3) && (\$29 == \$3) && (\$30 == \$3) && (\$35 == \$3) && (\$36 == \$3) && (\$41 == \$3) && (\$42 == \$3) && (\$47 == \$3) && (\$48 == \$3) && (\$53 == \$3) && (\$54 == \$4) && (\$56/(\$55+\$56) >= 0.25) && (\$56/(\$55+\$56) <= 0.75) && (\$26 == 0) && (\$32 == 0) && (\$38 == 0) && (\$44 == 0) && (\$50 == 0) ' $HOME_DIRECTORY/vcf_handling/VariantFiltration/"$a"_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_"$b"_PASS_splited_singletons_HOMREF_HET.table > $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$c"_"$b"_HOMREF_HET.table
## for sample6, HOM-VAR/HET
awk ' (\$23 == \$4) && (\$24 == \$4) && (\$29 == \$4) && (\$30 == \$4) && (\$35 == \$4) && (\$36 == \$4) && (\$41 == \$4) && (\$42 == \$4) && (\$47 == \$4) && (\$48 == \$4) && (\$53 == \$3) && (\$54 == \$4) && (\$56/(\$55+\$56) >= 0.25) && (\$56/(\$55+\$56) <= 0.75) && (\$25 == 0) && (\$31 == 0) && (\$37 == 0) && (\$43 == 0) && (\$49 == 0) ' $HOME_DIRECTORY/vcf_handling/VariantFiltration/"$a"_SNP_DPfiltered_AllCalled_BIALLELIC_VariantFiltration_"$b"_PASS_splited_singletons_HOMVAR_HET.table > $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$c"_"$b"_HOMVAR_HET.table
"; done > $HOME_DIRECTORY/scripts/singletons_filtering_sample6

#sh -e $HOME_DIRECTORY/scripts/singletons_filtering_sample1
#sh -e $HOME_DIRECTORY/scripts/singletons_filtering_sample2
#sh -e $HOME_DIRECTORY/scripts/singletons_filtering_sample3
#sh -e $HOME_DIRECTORY/scripts/singletons_filtering_sample4
#sh -e $HOME_DIRECTORY/scripts/singletons_filtering_sample5
#sh -e $HOME_DIRECTORY/scripts/singletons_filtering_sample6

<a id="39"></a> 
### Derive the number of candidate germline mutations for each sample

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu


touch $HOME_DIRECTORY/de_novo_mutation/consensus_approach_number_GM_before_manual_inspection

less $HOME_DIRECTORY/regime_parameter_type_sample_wide | while read a b c d e f g h i; do printf "
wc -l $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$d"_"$b"_"$c".table >> $HOME_DIRECTORY/de_novo_mutation/consensus_approach_number_GM_before_manual_inspection
wc -l $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$e"_"$b"_"$c".table >> $HOME_DIRECTORY/de_novo_mutation/consensus_approach_number_GM_before_manual_inspection
wc -l $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$f"_"$b"_"$c".table >> $HOME_DIRECTORY/de_novo_mutation/consensus_approach_number_GM_before_manual_inspection
wc -l $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$g"_"$b"_"$c".table >> $HOME_DIRECTORY/de_novo_mutation/consensus_approach_number_GM_before_manual_inspection
wc -l $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$h"_"$b"_"$c".table >> $HOME_DIRECTORY/de_novo_mutation/consensus_approach_number_GM_before_manual_inspection
wc -l $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$i"_"$b"_"$c".table >> $HOME_DIRECTORY/de_novo_mutation/consensus_approach_number_GM_before_manual_inspection
"; done > $HOME_DIRECTORY/scripts/HaplotypeCaller_number_GM_before_manual_inspection

sh -e $HOME_DIRECTORY/scripts/HaplotypeCaller_number_GM_before_manual_inspection

sed -i 's/\/gpfs\/home\/wwh23wcu\/de_novo_mutation\/consensus_approach\///' $HOME_DIRECTORY/de_novo_mutation/consensus_approach_number_GM_before_manual_inspection

sed -i 's/_/ /' $HOME_DIRECTORY/de_novo_mutation/consensus_approach_number_GM_before_manual_inspection
sed -i 's/_/ /' $HOME_DIRECTORY/de_novo_mutation/consensus_approach_number_GM_before_manual_inspection

sed -i 's/.table//' $HOME_DIRECTORY/de_novo_mutation/consensus_approach_number_GM_before_manual_inspection


awk -F' ' 'BEGIN { SUBSEP = OFS = FS } { s[$2,$3] += $1; ++c[$2,$3] } END { for (i in s) { print i, s[i], c[i] } }' $HOME_DIRECTORY/de_novo_mutation/consensus_approach_number_GM_before_manual_inspection | sort | uniq > $HOME_DIRECTORY/de_novo_mutation/consensus_approach_number_GM_before_manual_inspection_temp

mv $HOME_DIRECTORY/de_novo_mutation/consensus_approach_number_GM_before_manual_inspection_temp $HOME_DIRECTORY/de_novo_mutation/consensus_approach_number_GM_before_manual_inspection


<a id="40"></a> 
### Derive bed files for manual inspection
This script derives a bed file for each sample

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu


less $HOME_DIRECTORY/regime_sample | while read a b; do printf "
awk '{print \$1\" \"\$2\" \"\$3\" \"\$4}' $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$b"_1_HOMREF_HET.table > $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$b"_1_HOMREF_HET.bed
awk '{print \$1\" \"\$2\" \"\$4\" \"\$3}' $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$b"_1_HOMVAR_HET.table > $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$b"_1_HOMVAR_HET.bed

cat $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$b"_1_HOMREF_HET.bed $HOME_DIRECTORY/de_novo_mutation/consensus_approach/"$b"_1_HOMVAR_HET.bed > $HOME_DIRECTORY/de_novo_mutation/bed/"$b"_consensus_bed
"; done > $HOME_DIRECTORY/scripts/consensus_derive_bed

#sh -e $HOME_DIRECTORY/scripts/consensus_derive_bed

<a id="41"></a> 
## Callability

<a id="46"></a> 
### Derive genotyped sites for each regime
This script filters out sites with missing genotype in any of the samples of the same regime and keeps sites that all samples of the same regime are successfully genotyped.

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/callability
JOB_NAME=callability_cohort

echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=3G
#SBATCH --time=0-04:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

regime=$1

module load bcftools/1.15.1

bcftools view -i '"'"' FORMAT/GT[0] != "mis" && FORMAT/GT[1] != "mis" && FORMAT/GT[2] != "mis"  && FORMAT/GT[3] != "mis" && FORMAT/GT[4] != "mis" && FORMAT/GT[5] != "mis" '"'"' '$HOME_DIRECTORY'/vcf_handling/GenotypeGVCFs/$(basename $regime)_GenotypeGVCFs.g.vcf.gz \
--output '$HOME_DIRECTORY'/vcf_handling/callability/$(basename $regime)_GenotypeGVCFs_callable.g.vcf.gz

bcftools index -t '$HOME_DIRECTORY'/vcf_handling/callability/$(basename $regime)_GenotypeGVCFs_callable.g.vcf.gz

' > $HOME_DIRECTORY/scripts/callability_cohort.sh


less $HOME_DIRECTORY/regime_list | while read a b; do printf "
sbatch $HOME_DIRECTORY/scripts/callability_cohort.sh "$a"
"; done > $HOME_DIRECTORY/scripts/callability_cohort_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/callability_cohort_submit_jobs


<a id="43"></a> 
### Derive the number of callable sites for each regime

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu


touch $HOME_DIRECTORY/de_novo_mutation/consensus_approach_callability_regime

less $HOME_DIRECTORY/regime_list | while read a; do printf "
bcftools index --nrecords $HOME_DIRECTORY/vcf_handling/callability/"$a"_GenotypeGVCFs_callable.g.vcf.gz >> $HOME_DIRECTORY/de_novo_mutation/consensus_approach_callability_regime
"; done > $HOME_DIRECTORY/scripts/callability_cohort_derive_results


sed -i '1i module load bcftools/1.15.1' $HOME_DIRECTORY/scripts/callability_cohort_derive_results

# sh -e $HOME_DIRECTORY/scripts/callability_cohort_derive_results


<a id="44"></a> 
# Variant calling by the probabilistic approach

<a id="45"></a> 
## Merge bam files for each regime
This script uses the picard MergeSamFiles tool to merge single-sample bam files of the same regime into a multi-sample bam file. Total three multi-sample bam files, one for each regime, are generated.

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output
JOB_NAME=picard_MergeSamFiles

echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=60G
#SBATCH --time=0-48:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

regime=$1
regime_sample1=$2
regime_sample2=$3
regime_sample3=$4
regime_sample4=$5
regime_sample5=$6
regime_sample6=$7

module load java/jdk1.8.0_231
module load picard/2.24.1


java -jar /gpfs/software/ada/picard/2.24.1/picard.jar MergeSamFiles \
I='$SCRATCH_DIRECTORY'/bam_files/$(basename $regime_sample1)_viewed_marked_addg.bam \
I='$SCRATCH_DIRECTORY'/bam_files/$(basename $regime_sample2)_viewed_marked_addg.bam \
I='$SCRATCH_DIRECTORY'/bam_files/$(basename $regime_sample3)_viewed_marked_addg.bam \
I='$SCRATCH_DIRECTORY'/bam_files/$(basename $regime_sample4)_viewed_marked_addg.bam \
I='$SCRATCH_DIRECTORY'/bam_files/$(basename $regime_sample5)_viewed_marked_addg.bam \
I='$SCRATCH_DIRECTORY'/bam_files/$(basename $regime_sample6)_viewed_marked_addg.bam \
O='$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles.bam

' > $HOME_DIRECTORY/scripts/MergeSamFiles.sh


less $HOME_DIRECTORY/regime_sample_wide | while read a b c d e f g; do printf "
sbatch $HOME_DIRECTORY/scripts/MergeSamFiles.sh "$a" "$b" "$c" "$d" "$e" "$f" "$g"
"; done > $HOME_DIRECTORY/scripts/MergeSamFiles_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/MergeSamFiles_submit_jobs


<a id="46"></a> 
## Add a readgroup for the ancestor to the header of the merged bam
This script uses the samtools view tool to add a dummy readgroup for the ancestor to the merged, multi-sample bam file.

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/accuMUlate
JOB_NAME=addg


echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=12G
#SBATCH --time=0-12:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

regime=$1


module load samtools/1.16.1

samtools view -H '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles.bam > '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles.head.sam

sed '"'"'19 i @RG\tID:Ancestral\tLB:lib1 PL:illumina_TruSeq\tSM:Ancestral\tPU:unit1'"'"' '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles.head.sam > '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles_addg.head.sam

samtools view '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles.bam | cat '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles_addg.head.sam - | samtools view -Sb - > '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles_addg.bam

samtools index '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles_addg.bam


module load python/3.10.0

samtools view -H '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles_addg.bam | python '$HOME_DIRECTORY'/extract_samples.py Ancestral - >> '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles_params.ini


cat '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles_params.ini '$HOME_DIRECTORY'/accuMUlate_parameters > '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles_params_temp.ini && mv '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles_params_temp.ini '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles_params.ini

' > $HOME_DIRECTORY/scripts/addg.sh



less $HOME_DIRECTORY/regime_list | while read a; do printf "
sbatch $HOME_DIRECTORY/scripts/addg.sh "$a"
"; done > $HOME_DIRECTORY/scripts/addg_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/addg_submit_jobs




<a id="47"></a> 
## Call variants
This script uses the accuMUlate tool to call variants.

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
REF_FASTA_DECOMPRESSED=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/accuMUlate
JOB_NAME=accuMUlate

echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=30G
#SBATCH --time=5-00:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

regime=$1

module load accuMUlate/0.2.2

/gpfs/software/ada/accuMUlate/0.2.2/accuMUlate \
-b '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles_addg.bam \
-r '$REF_FASTA_DECOMPRESSED' \
-q 13 \
-m 13 \
-p 0.10000000000000001 \
--ploidy-ancestor 2 \
--ploidy-descendant 2 \
-c '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles_params.ini \
-o '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_putative_mutations.tsv

' > $HOME_DIRECTORY/scripts/accuMUlate.sh


less $HOME_DIRECTORY/regime_list | while read a b c d e f g; do printf "
sbatch $HOME_DIRECTORY/scripts/accuMUlate.sh "$a"
"; done > $HOME_DIRECTORY/scripts/accuMUlate_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/accuMUlate_submit_jobs


<a id="48"></a> 
## Variant filtering
This scripts filters to keep the following:
1. Sites outside of soft-masked regions;
2. Sites where estimated probability of at least one mutation at this site = 1;
3. Sites where estimated probability of exactly one mutation at this site = 1; 
4. Sites where probability of the mutation direction above being correct = 1; 
5. Homozygous to heterozygous mutations;
6. Sites where the number of reads for the mutant allele in non-mutant samples = 0; 
7. Sites where the allelic ratio for the mutant allele in the mutant sample is between 0.25 and 0.75;
8. Sites where the AD test statistic for mapping quality difference between mutant and non-mutant alleles =< 1.95;
9. Sites where the AD test statistic for insert size differfence between inferred insert size between mutant and non-mutant alleles =< 1.95;
10. Sites where the p-value from Fisher's exact test of strand bias between mutant and non-mutant alleles is > 0.05;
11. Sites where the p-value from Fisher's exact test of pair-mapping rate difference between mutant and non-mutant alleles is > 0.05.

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

less $HOME_DIRECTORY/regime_list | while read a; do printf "

awk '"' $4~/[A-Z]/ '"' $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations.tsv > $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp1.tsv

awk '"' $7==1 && $8==1 && $9==1 {print $0}'"' $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp1.tsv > $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp2.tsv

awk '"' { if (substr($6,1,1) == substr($6,2,1) && substr($6,1,1) == substr($6,5,1) && substr($6,5,1) != substr($6,6,1) ) print $0 }'"' $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp2.tsv > $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp3.tsv

awk '"' $15==0 {print $0}'"' $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp3.tsv > $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp4.tsv

awk '"' $14/($14+$12+$13) >=0.25 && $14/($14+$12+$13) <= 0.75 {print $0}'"' $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp4.tsv > $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp5.tsv

awk '"' $16 <= 1.95 {print $0} '"' $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp5.tsv > $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp6.tsv

awk '"' $17 <= 1.95 {print $0} '"' $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp6.tsv > $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp7.tsv

awk '"' $18 > 0.05 {print $0} '"' $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp7.tsv > $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp8.tsv

awk '"' $19 > 0.05 {print $0} '"' $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp8.tsv > $HOME_DIRECTORY/de_novo_mutation/probabilistic_approach/"$a"_filtered_mutations.tsv

rm $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp1.tsv
rm $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp2.tsv
rm $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp3.tsv
rm $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp4.tsv
rm $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp5.tsv
rm $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp6.tsv
rm $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp7.tsv
rm $HOME_DIRECTORY/accuMUlate/"$a"_putative_mutations_temp8.tsv
"; done > $HOME_DIRECTORY/scripts/accuMUlate_filtering

#sh -e $HOME_DIRECTORY/scripts/accuMUlate_filtering


<a id="49"></a> 
## Derive the number of candidate germline mutations for each sample

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

touch $HOME_DIRECTORY/de_novo_mutation/probabilistic_approach_number_GM

less $HOME_DIRECTORY/regime_sample | while read a b; do printf "
grep '"$b"' $HOME_DIRECTORY/de_novo_mutation/probabilistic_approach/"$a"_filtered_mutations.tsv > $HOME_DIRECTORY/de_novo_mutation/probabilistic_approach/"$b"_filtered_mutations.tsv
wc -l $HOME_DIRECTORY/de_novo_mutation/probabilistic_approach/"$b"_filtered_mutations.tsv >> $HOME_DIRECTORY/de_novo_mutation/probabilistic_approach_number_GM_before_manual_inspection
"; done > $HOME_DIRECTORY/scripts/accuMUlate_number_GM_before_manual_inspection

sh -e $HOME_DIRECTORY/scripts/accuMUlate_number_GM_before_manual_inspection

sed -i 's/\/gpfs\/home\/wwh23wcu\/de_novo_mutation\/probabilistic_approach\///' $HOME_DIRECTORY/de_novo_mutation/probabilistic_approach_number_GM_before_manual_inspection

sed -i 's/_filtered_mutations.tsv//' $HOME_DIRECTORY/de_novo_mutation/probabilistic_approach_number_GM_before_manual_inspection


<a id="50"></a> 
## Derive denominator
This script uses the accuMUlate denominate tool to derive the denominator for mutation rate calculation.

In [ ]:
## Derive denominator

HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
REF_FASTA_DECOMPRESSED=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/accuMUlate
JOB_NAME=denominator

echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=30G
#SBATCH --time=5-00:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

regime=$1

module load accuMUlate/0.2.2

accuMUlate denominate -c '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles_params.ini -r '$REF_FASTA_DECOMPRESSED' -b '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_MergeSamFiles_addg.bam > '$HOME_DIRECTORY'/accuMUlate/$(basename $regime)_denom.out

' > $HOME_DIRECTORY/scripts/denominator.sh


less $HOME_DIRECTORY/regime_list | while read a; do printf "
sbatch $HOME_DIRECTORY/scripts/denominator.sh "$a"
"; done > $HOME_DIRECTORY/scripts/denominator_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/denominator_submit_jobs



<a id="51"></a> 
## Derive bed files for manual inspection
This script derives a bed file for each sample

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

less $HOME_DIRECTORY/regime_sample | while read a b; do echo "

awk '{print \$1\" \"\$3\" \"\$4\" \"substr(\$6,6,1)}' $HOME_DIRECTORY/de_novo_mutation/probabilistic_approach/"$b"_filtered_mutations.tsv > $HOME_DIRECTORY/de_novo_mutation/bed/"$b"_probabilistic_bed

"; done > $HOME_DIRECTORY/scripts/probabilistic_derive_bed

sh -e $HOME_DIRECTORY/scripts/probabilistic_derive_bed


<a id="52"></a> 
# Manual inspection

<a id="53"></a> 
## Merge and annotate the bed files from both approaches for each sample

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

less $HOME_DIRECTORY/regime_sample | while read a b; do echo "
awk '\$5 = \"consensus\" {print \$1\" \"\$2\" \"\$3\" \"\$4\" \"\$5}' $HOME_DIRECTORY/de_novo_mutation/bed/"$b"_consensus_bed > $HOME_DIRECTORY/de_novo_mutation/bed/"$b"_consensus_bed_temp
awk '\$5 = \"probabilistic\" {print \$1\" \"\$2\" \"\$3\" \"\$4\" \"\$5}' $HOME_DIRECTORY/de_novo_mutation/bed/"$b"_probabilistic_bed > $HOME_DIRECTORY/de_novo_mutation/bed/"$b"_probabilistic_bed_temp

cat $HOME_DIRECTORY/de_novo_mutation/bed/"$b"_consensus_bed_temp $HOME_DIRECTORY/de_novo_mutation/bed/"$b"_probabilistic_bed_temp > $HOME_DIRECTORY/de_novo_mutation/bed/"$b"_bed_temp

sort $HOME_DIRECTORY/de_novo_mutation/bed/"$b"_bed_temp > $HOME_DIRECTORY/de_novo_mutation/bed/manual_inspection/"$b"_bed_before_manual_inspection

rm $HOME_DIRECTORY/de_novo_mutation/bed/*_temp
"; done > $HOME_DIRECTORY/scripts/prepare_bed_files

sh -e $HOME_DIRECTORY/scripts/prepare_bed_files


<a id="54"></a> 
## Derive single-sample vcf
This script splits the multi-sample vcf derived from joint genotyping to six single-sample vcf

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/bcftools_split
JOB_NAME=bcftools_split

echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=3G
#SBATCH --time=0-04:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

regime=$1

module load bcftools/1.15.1
module load htslib/1.15.1

bcftools +split '$HOME_DIRECTORY'/vcf_handling/GenotypeGVCFs/$(basename $regime)_GenotypeGVCFs.g.vcf.gz -O z -o '$HOME_DIRECTORY'/de_novo_mutation/bed/manual_inspection/

' > $HOME_DIRECTORY/scripts/bcftools_split.sh


less $HOME_DIRECTORY/regime_list | while read a; do printf "
sbatch $HOME_DIRECTORY/scripts/bcftools_split.sh "$a"
"; done > $HOME_DIRECTORY/scripts/bcftools_split_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/bcftools_split_submit_jobs


<a id="55"></a> 
## Index single-sample vcf
This script indexes the single-sample vcf files

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

REF_FASTA=$SCRATCH_DIRECTORY/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.gz
JOB_OUTPUT=$SCRATCH_DIRECTORY/job_output/bcftools_split
JOB_NAME=bcftools_split_tabix

echo '#!/bin/bash -e
#SBATCH --mail-type=ALL #Mail events (NONE, BEGIN, END, FAIL, ALL) 
#SBATCH --mail-user=papilio.chen@gmail.com #Where to send mail
#SBATCH -p compute-24-96
#SBATCH --mem=3G
#SBATCH --time=0-04:00
#SBATCH --job-name='$JOB_NAME'
#SBATCH -o '$JOB_OUTPUT'/'$JOB_NAME'-%j.out
#SBATCH -e '$JOB_OUTPUT'/'$JOB_NAME'-%j.err

#ABOVE THIS LINE ARE THE SLURM DIRECTIVES
#BELOW THIS LINE ARE THE COMMANDS TO RUN YOUR JOB

regime=$1
sample_ID=$2

module load bcftools/1.15.1
module load htslib/1.15.1

tabix -p vcf '$HOME_DIRECTORY'/de_novo_mutation/bed/manual_inspection/$(basename $sample_ID).vcf.gz

' > $HOME_DIRECTORY/scripts/bcftools_split_tabix.sh


less $HOME_DIRECTORY/regime_sample_field_for_DP | while read a b c; do printf "
sbatch $HOME_DIRECTORY/scripts/bcftools_split_tabix.sh "$a" "$b"
"; done > $HOME_DIRECTORY/scripts/bcftools_split_tabix_submit_jobs

# sh -e $HOME_DIRECTORY/scripts/bcftools_split_tabix_submit_jobs


<a id="56"></a> 
## Files needed for manual inspection

Files needed for manual inspection include:
1 and 2: the reference assembly;
3 and 4: bam and bai files for each sample;
5 and 6: single-sample vcf and its index for each sample;
7: bed files containing the coordination for each candidate germline mutation for each sample.

1. /gpfs/scratch/wwh23wcu/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna
2. /gpfs/scratch/wwh23wcu/ref_fasta/GCA_001643735.4_ASM164373v4_genomic.fna.fai
3. /gpfs/scratch/wwh23wcu/bam_files/*_viewed_marked_addg.bam 
4. /gpfs/scratch/wwh23wcu/bam_files/*_viewed_marked_addg.bam.bai
5. /gpfs/home/wwh23wcu/de_novo_mutation/bed/manual_inspection/*.vcf.gz
6. /gpfs/home/wwh23wcu/de_novo_mutation/bed/manual_inspection/*.vcf.gz.tbi
7. /gpfs/home/wwh23wcu/de_novo_mutation/bed/manual_inspection/*_bed_before_manual_inspection

For each sample, load the reference assembly and the vcf in IGV; then load the bam files for all samples of the same regime in IGV. Manually inspect individual candidate germline mutation and annotate 'ACCEPTED' or 'REJECTED' in the bed file. When finished, save the bed file as sampleID_bed_after_manual_inspection, and upload it to /gpfs/home/wwh23wcu/de_novo_mutation/bed/manual_inspection/.

<a id="57"></a> 
# Find overlapping variants

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

less $HOME_DIRECTORY/regime_sample | while read a b; do echo "
awk '{print \$1\" \"\$2\" \"\$3\" \"\$4}' $HOME_DIRECTORY/de_novo_mutation/bed/manual_inspection/"$b"_bed_after_manual_inspection | sort | uniq -d > $HOME_DIRECTORY/de_novo_mutation/"$b"_bed_after_manual_inspection_overlapping
"; done > $HOME_DIRECTORY/scripts/find_overlapping_variants

<a id="58"></a> 
# Aggregate the results
This script aggregates callability and the number of accepted germline mutations for each sample.

In [ ]:
HOME_DIRECTORY=/gpfs/home/wwh23wcu
SCRATCH_DIRECTORY=/gpfs/scratch/wwh23wcu

paste -d ' ' $HOME_DIRECTORY/regime_list $HOME_DIRECTORY/de_novo_mutation/consensus_approach_callability_regime > $HOME_DIRECTORY/de_novo_mutation/consensus_approach_callability_regime_temp && mv $HOME_DIRECTORY/de_novo_mutation/consensus_approach_callability_regime_temp $HOME_DIRECTORY/de_novo_mutation/consensus_approach_callability_regime

perl -ne 'print $_ x 36' $HOME_DIRECTORY/de_novo_mutation/consensus_approach_callability_regime > $HOME_DIRECTORY/de_novo_mutation/consensus_approach_callability_regime_temp

paste -d ' ' $HOME_DIRECTORY/de_novo_mutation/consensus_approach_callability_regime_temp $HOME_DIRECTORY/de_novo_mutation/consensus_approach_number_GM > $HOME_DIRECTORY/de_novo_mutation/consensus_approach_results_temp

awk '{$6=""; print $0}' $HOME_DIRECTORY/de_novo_mutation/consensus_approach_results_temp > $HOME_DIRECTORY/de_novo_mutation/consensus_approach_results


rm $HOME_DIRECTORY/de_novo_mutation/consensus_approach_callability_regime_temp
rm $HOME_DIRECTORY/de_novo_mutation/consensus_approach_results_temp